In [0]:
#@title Setup

%cd 'drive/My Drive/NLP_project'
!pip install pyvi

from preprocessing import NLP
from VectorizationWord import FeatureExtraction
from tensorflow.keras.models import load_model
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from glob import glob
import numpy as np
from define import *
from gensim.models import Word2Vec

word2vec = Word2Vec.load(WORD2VEC_PATH)
tfidf = pickle.load(open(TFIDF_PATH, "rb"))
tfidf_weighed = pickle.load(open(TFIDF_PATH.replace('.p','weighed.p'), "rb"))

lb = pickle.load(open('features/LabelEncoder.p', "rb"))

h5paths_tfidf = glob('model/bestcp_TFIDF7_*.h5')
model_tfidf = load_model(h5paths_tfidf[-1])
h5paths_w2v = glob('model/bestcp_W2V3_*.h5')
model_w2v = load_model(h5paths_w2v[-1])
h5paths_wwv = glob('model/bestcp_WWV5_*.h5')
model_wwv = load_model(h5paths_wwv[-1])

In [11]:
#@title Phân loại văn bản {display-mode: "form"}

Input = "HLV Mallorca s\u1EE3 va ch\u1EA1m v\u1EDBi Messi"  #@param {type: "string"}

Method = "Weighted Word Vectors" #@param ['TF-IDF', 'Word2Vec', 'Weighted Word Vectors']
#@markdown ---

method_dict = {
    'TF-IDF' : 0,
    'Word2Vec' : 1,
    'Weighted Word Vectors' : 2
}

VectorizationWordMethod = method_dict[Method]

data = NLP(Input).get_text_feature()

if VectorizationWordMethod == 0:
    features = tfidf.transform([data]).todense()
    y_pred = model_tfidf.predict(features)
    label = lb.inverse_transform([np.argmax(y_pred)])

elif VectorizationWordMethod == 1:
    words = data.split(" ")
    vec = []
    for i in range(40):
        try:
            vec.append(word2vec.wv[words[i]])
        except KeyError:
            vec.append(np.random.normal(0, 0.05, 300))
        except IndexError:
            vec.append(np.random.normal(0, 0.05, 300))
    features = np.asarray(vec).reshape(1, 40, 300)
    y_pred = model_w2v.predict(features)
    label = lb.inverse_transform([np.argmax(y_pred)])
    
elif VectorizationWordMethod == 2:
    d_tfidf = tfidf_weighed.transform([data])
    vocal = tfidf_weighed.vocabulary_
    words = data.split()
    vecs = []
    for i in range(40):
        try:
            w2v = np.asarray(word2vec.wv[words[i]])
        except KeyError:
            w2v = np.random.normal(0, 0.05, 300)
        except IndexError:
            w2v = np.random.normal(0, 0.05, 300)
        try:
            tfidf = d_tfidf[0, vocal[words[i]]]
        except KeyError:
            tfidf = np.random.normal(0, 0.05, 1)[0]
        except IndexError:
            tfidf = np.random.normal(0, 0.05, 1)[0]
        vec = tfidf * w2v
        vecs.append(vec)
    features = np.asarray(vecs).reshape(1, 40, 300)
    y_pred = model_wwv.predict(test_data)
    label = lb.inverse_transform([np.argmax(y_pred)])
else:
    pass

print(label[0])

thể thao
